Author: Dana Chermesh Reshef, DRAW Brooklyn<br>
April 2019

### _Digital CEQR -- #4_
# Data: Labor force
- **_ACS 5-yesr estimates 2013-2017 using Census API_**
- **_ACS 5-yesr estimates 2006-2010 using Census API_**

----

** A user guide for Census Data API can be found in [notebook no.3: data_3-Housing_byTenure_ACS5](https://github.com/drawbrooklyn/digitalceqr/blob/master/data_3-Housing_byTenure_ACS5.ipynb)

----
## Labor Force 2017
### _data were obtained from the ACS 2013-2017 5-year estimate, all counties in the US_

[list of variables](https://api.census.gov/data/2017/acs/acs5/variables)

variables to be acquired:
- **B23025_001E** |	All pop at age 16 and over
- **B23025_007E** | All pop at age 16 and over, not in labor force

for prime age (25-54):
- **B23001_025E** | Male 25 to 29 in labor force
- **B23001_032E** | Male 30 to 34 in labor force
- **B23001_039E** | Male 35 to 44 in labor force
- **B23001_046E** | Male 45 to 54 in labor force
- **B23001_111E** | Female 25 to 29 in labor force
- **B23001_118E** | Female 30 to 34 in labor force
- **B23001_125E** | Female 35 to 44 in labor force
- **B23001_132E** | Female 45 to 54 in labor force

for over 65:
- **B23001_074E** | Male 65 to 69 in labor force
- **B23001_079E** | Male 70 to 74 in labor force
- **B23001_084E** | Male 75 and over in labor force
- **B23001_160E** | Female 65 to 69 in labor force
- **B23001_165E** | Female 70 to 74 in labor force
- **B23001_170E** | Female 75 and over in labor force

for 55-64:
- **B23001_053E** | Male 54 to 59 in labor force
- **B23001_060E** | Male 60 to 61 in labor force
- **B23001_067E** | Male 62 and 64 in labor force
- **B23001_139E** | Female 54 to 59 in labor force
- **B23001_146E** | Female 60 to 61 in labor force
- **B23001_153E** | Female 62 and 64 in labor force

In [1]:
import pandas as pd
import json
# reading in my api key saved in censusAPI.py as
# myAPI = 'XXXXXXXXXXXXXXX'
# request an api key in: https://api.census.gov/data/key_signup.html
from censusAPI import myAPI

In [2]:
import json
import requests 
import urllib
import numpy as np

#read in in the variables available. the info you need is in the 1year ACS data
url = "https://api.census.gov/data/2017/acs/acs5/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

In [3]:
#turning things into arrays to enable broadcasting
#Python3
affkeys = np.array(list(aff1y['variables'].keys()))

affkeys

array(['B20004_012E', 'C23002A_010E', 'B08126_070E', ..., 'B26201_026E',
       'B24092_013E', 'C23002D_002E'], dtype='<U14')

In [4]:
print(aff1y['variables']['B23025_001E'])
print(aff1y['variables']['B23025_007E'])

{'predicateType': 'int', 'group': 'B23025', 'concept': 'EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER', 'limit': 0, 'label': 'Estimate!!Total', 'attributes': 'B23025_001M,B23025_001MA,B23025_001EA'}
{'predicateType': 'int', 'group': 'B23025', 'concept': 'EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER', 'limit': 0, 'label': 'Estimate!!Total!!Not in labor force', 'attributes': 'B23025_007M,B23025_007MA,B23025_007EA'}


In [6]:
Labor17vars = ['B23025_001E', 'B23025_007E', 'B23001_025E', 'B23001_032E',
               'B23001_039E', 'B23001_046E', 'B23001_053E', 'B23001_060E',
               'B23001_067E', 'B23001_139E', 'B23001_146E', 'B23001_111E',
               'B23001_153E', 'B23001_118E', 'B23001_125E', 'B23001_132E', 
               'B23001_074E', 'B23001_079E', 'B23001_084E', 'B23001_160E',
               'B23001_165E', 'B23001_170E']

Labor17str = ",".join(Labor17vars)
Labor17str

'B23025_001E,B23025_007E,B23001_025E,B23001_032E,B23001_039E,B23001_046E,B23001_053E,B23001_060E,B23001_067E,B23001_139E,B23001_146E,B23001_111E,B23001_153E,B23001_118E,B23001_125E,B23001_132E,B23001_074E,B23001_079E,B23001_084E,B23001_160E,B23001_165E,B23001_170E'

In [7]:
# Labor Force data for all counties in the US, 2017
Labor17 = pd.read_json('https://api.census.gov/data/2017/acs/acs5?get='+
                          Labor17str +
                         ',NAME&for=county:*&in=state:*')
Labor17.columns = Labor17.iloc[0]
Labor17 = Labor17[1:]

Labor17['state'] = Labor17['state'].apply(lambda x: '{0:0>2}'.format(x))
Labor17['county'] = Labor17['county'].apply(lambda x: '{0:0>3}'.format(x))
Labor17['STCO'] = Labor17[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

# converting dtypes to int
for col in Labor17vars:
    Labor17[col] = Labor17[col].astype(int)

# aggregating age groups and calculations
Labor17['LaborForce'] = Labor17['B23025_001E'] - Labor17['B23025_007E']
Labor17['PrimeAge'] = Labor17['B23001_025E'] + Labor17['B23001_032E'] +\
                      Labor17['B23001_039E'] + Labor17['B23001_046E'] +\
                      Labor17['B23001_111E'] + Labor17['B23001_118E'] +\
                      Labor17['B23001_125E'] + Labor17['B23001_132E']
Labor17['Over65'] = Labor17['B23001_074E'] + Labor17['B23001_079E'] +\
                    Labor17['B23001_084E'] + Labor17['B23001_160E'] +\
                    Labor17['B23001_165E'] + Labor17['B23001_170E']
Labor17['55-64'] = Labor17['B23001_053E'] + Labor17['B23001_060E'] +\
                   Labor17['B23001_067E'] + Labor17['B23001_139E'] +\
                   Labor17['B23001_146E'] + Labor17['B23001_153E']
Labor17['Under25'] = Labor17['LaborForce'] - Labor17['PrimeAge'] -\
                     Labor17['Over65'] - Labor17['55-64']

Labor17 = Labor17.drop(['state', 'county', 'B23025_001E', 'B23025_007E',
                        'B23001_025E', 'B23001_032E', 'B23001_039E',
                        'B23001_046E', 'B23001_111E', 'B23001_118E', 
                        'B23001_125E', 'B23001_132E', 'B23001_074E',
                        'B23001_079E', 'B23001_084E', 'B23001_160E',
                        'B23001_139E', 'B23001_146E', 'B23001_153E',
                        'B23001_053E', 'B23001_060E', 'B23001_067E',
                        'B23001_165E', 'B23001_170E'], axis=1)
Labor17.columns = ['Name', 'STCO', 'LaborForce', 'PrimeAge', 
                   'Over65', '55-64', 'Under25']

print(Labor17.shape)
print(Labor17.dtypes)
Labor17.head()

(3220, 7)
Name          object
STCO          object
LaborForce     int64
PrimeAge       int64
Over65         int64
55-64          int64
Under25        int64
dtype: object


,Name,STCO,LaborForce,PrimeAge,Over65,55-64,Under25
1,"Corozal Municipio, Puerto Rico",72047,11323,8898,103,962,1360
2,"Maunabo Municipio, Puerto Rico",72095,2991,2196,90,328,377
3,"Peñuelas Municipio, Puerto Rico",72111,6895,5506,212,638,539
4,"Ponce Municipio, Puerto Rico",72113,45928,32200,2010,6747,4971
5,"San Sebastián Municipio, Puerto Rico",72131,11829,8855,230,1185,1559


----
## Labor Force 2000
### _data were obtained from US Census Bureau Decennial 2000 Census, SF3_

[list of variables](https://api.census.gov/data/2000/sf3/variables.html)

variables to be acquired:
- **P043003** | Male In labor force
- **P043010** | Female In labor force

for prime age (25-54):
- **PCT035025** | Male 25 to 29 in labor force
- **PCT035032** | Male 30 to 34 in labor force
- **PCT035039** | Male 35 to 44 in labor force
- **PCT035046** | Male 45 to 54 in labor force
- **PCT035117** | Female 25 to 29 in labor force
- **PCT035124** | Female 30 to 34 in labor force
- **PCT035131** | Female 35 to 44 in labor force
- **PCT035138** | Female 45 to 54 in labor force

for over 65:
- **PCT035074** | Male 65 to 69 in labor force
- **PCT035081** | Male 70 to 74 in labor force
- **PCT035088** | Male 75 and over in labor force
- **PCT035166** | Female 65 to 69 in labor force
- **PCT035173** | Female 70 to 74 in labor force
- **PCT035180** | Female 75 and over in labor force

for 55-64:
- **PCT035053** | Male 55 to 59 in labor force
- **PCT035060** | Male 60 to 61 in labor force
- **PCT035067** | Male 62 to 64 in labor force
- **PCT035145** | Female 55 to 59 in labor force
- **PCT035152** | Female 60 to 61 in labor force
- **PCT035159** | Female 62 to 64 in labor force

In [8]:
Labor00vars = ['P043003', 'P043010', 'PCT035025', 'PCT035032',
               'PCT035039', 'PCT035046', 'PCT035053', 'PCT035060',
               'PCT035067', 'PCT035145', 'PCT035152', 'PCT035159',
               'PCT035117', 'PCT035124', 'PCT035131', 'PCT035138',
               'PCT035074', 'PCT035081', 'PCT035088', 'PCT035166', 
               'PCT035173', 'PCT035180']

Labor00str = ",".join(Labor00vars)
Labor00str

'P043003,P043010,PCT035025,PCT035032,PCT035039,PCT035046,PCT035053,PCT035060,PCT035067,PCT035145,PCT035152,PCT035159,PCT035117,PCT035124,PCT035131,PCT035138,PCT035074,PCT035081,PCT035088,PCT035166,PCT035173,PCT035180'

In [9]:
# Labor Force data for all counties in the US, 2000
Labor00 = pd.read_json('https://api.census.gov/data/2000/sf3?get='+
                        Labor00str +
                        ',NAME&for=county:*&in=state:*')
Labor00.columns = Labor00.iloc[0]
Labor00 = Labor00[1:]

Labor00['state'] = Labor00['state'].apply(lambda x: '{0:0>2}'.format(x))
Labor00['county'] = Labor00['county'].apply(lambda x: '{0:0>3}'.format(x))
Labor00['STCO'] = Labor00[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

# converting dtypes to int
for col in Labor00vars:
    Labor00[col] = Labor00[col].astype(int)

Labor00['LaborForce00'] = Labor00['P043003'] + Labor00['P043010']
Labor00['PrimeAge00'] = Labor00['PCT035025'] + Labor00['PCT035032'] +\
                      Labor00['PCT035039'] + Labor00['PCT035046'] +\
                      Labor00['PCT035117'] + Labor00['PCT035124'] +\
                      Labor00['PCT035131'] + Labor00['PCT035138']
Labor00['Over65_00'] = Labor00['PCT035074'] + Labor00['PCT035081'] +\
                       Labor00['PCT035088'] + Labor00['PCT035166'] +\
                       Labor00['PCT035173'] + Labor00['PCT035180']
Labor00['55-64_00'] = Labor00['PCT035053'] + Labor00['PCT035060'] +\
                      Labor00['PCT035067'] + Labor00['PCT035145'] +\
                      Labor00['PCT035152'] + Labor00['PCT035159']
Labor00['Under25_00'] = Labor00['LaborForce00'] - Labor00['PrimeAge00'] -\
                        Labor00['Over65_00'] - Labor00['55-64_00']

Labor00 = Labor00.drop(['state', 'county', 'P043003', 'P043010',
                        'PCT035025', 'PCT035032', 'PCT035039',
                        'PCT035046', 'PCT035117', 'PCT035124', 
                        'PCT035131', 'PCT035138', 'PCT035074',
                        'PCT035081', 'PCT035088', 'PCT035166',
                        'PCT035053', 'PCT035060', 'PCT035067',
                        'PCT035145', 'PCT035152', 'PCT035159',
                        'PCT035173', 'PCT035180'], axis=1)
Labor00.columns = ['Name', 'STCO', 'LaborForce00', 'PrimeAge00',
                   'Over65_00', '55-64_00', 'Under25_00']

print(Labor00.shape)
print(Labor00.dtypes)
Labor00.head()

(3219, 7)
Name            object
STCO            object
LaborForce00     int64
PrimeAge00       int64
Over65_00        int64
55-64_00         int64
Under25_00       int64
dtype: object


,Name,STCO,LaborForce00,PrimeAge00,Over65_00,55-64_00,Under25_00
1,Autauga County,01001,21167,15026,588,2262,3291
2,Baldwin County,01003,65960,46800,2499,7771,8890
3,Barbour County,01005,10826,7656,473,1130,1567
4,Bibb County,01007,8521,6144,204,917,1256
5,Blount County,01009,23896,16867,832,2869,3328


## Merging 2000 and 2017 data

In [10]:
Labor = Labor00.merge(Labor17, on='STCO')

Labor = Labor.drop(['Name_x'], axis=1)
Labor = Labor.append(Labor.sum(numeric_only=True), ignore_index=True)
Labor = Labor.set_index('Name_y').fillna(0).astype(int)

Labor['over65_%_16'] = Labor['Over65'] / Labor['LaborForce']
Labor['over65_%_00'] = Labor['Over65_00'] / Labor['LaborForce00']

Labor['over65_%change'] = Labor['over65_%_16'] - Labor['over65_%_00']

Labor['NET_LaborTotal'] = Labor['LaborForce'] - Labor['LaborForce00']
Labor['%_LaborTotal'] = (Labor['LaborForce'] - Labor['LaborForce00']) \
                         / Labor['LaborForce00']
Labor['NET_PrimeAge'] = Labor['PrimeAge'] - Labor['PrimeAge00']
Labor['%_PrimeAge'] = (Labor['PrimeAge'] - Labor['PrimeAge00']) \
                      / Labor['PrimeAge00']
Labor['NET_Over65'] = Labor['LaborForce'] - Labor['LaborForce00']
Labor['%_Over65'] = (Labor['Over65'] - Labor['Over65_00']) \
                     / Labor['Over65_00']
Labor['NET_55-64'] = Labor['55-64'] - Labor['55-64_00']
Labor['%_55-64'] = (Labor['55-64'] - Labor['55-64_00']) \
                      / Labor['55-64_00']
Labor['NET_Under25'] = Labor['Under25'] - Labor['Under25_00']
Labor['%_Under25'] = (Labor['Under25'] - Labor['Under25_00']) \
                     / Labor['Under25_00']


Labor.index = Labor.index.fillna('UStotal')

print(Labor.shape)
Labor.tail()

(3213, 24)


,STCO,LaborForce00,PrimeAge00,Over65_00,55-64_00,Under25_00,LaborForce,PrimeAge,Over65,55-64,...,NET_LaborTotal,%_LaborTotal,NET_PrimeAge,%_PrimeAge,NET_Over65,%_Over65,NET_55-64,%_55-64,NET_Under25,%_Under25
Name_y,,,,,,,,,,,,,,,,,,,,,
"Vieques Municipio, Puerto Rico",72147,2395,1657,33,343,362,3355,2091,168,643,...,960,0.400835,434,0.261919,960,4.090909,300,0.874636,91,0.251381
"Villalba Municipio, Puerto Rico",72149,7463,5547,87,355,1474,9166,6341,173,1043,...,1703,0.228192,794,0.143140,1703,0.988506,688,1.938028,135,0.091588
"Yabucoa Municipio, Puerto Rico",72151,9498,6812,52,477,2157,10592,7633,112,1023,...,1094,0.115182,821,0.120523,1094,1.153846,546,1.144654,-333,-0.154381
"Yauco Municipio, Puerto Rico",72153,13622,10020,163,1076,2363,11842,8886,344,1512,...,-1780,-0.130671,-1134,-0.113174,-1780,1.110429,436,0.405204,-1263,-0.534490
UStotal,0,139958254,98706665,4663882,14095193,22492514,163394880,105259275,8387451,26439530,...,23436626,0.167454,6552610,0.066385,23436626,0.798384,12344337,0.875783,816110,0.036284


In [11]:
Labor.dtypes

STCO                int64
LaborForce00        int64
PrimeAge00          int64
Over65_00           int64
55-64_00            int64
Under25_00          int64
LaborForce          int64
PrimeAge            int64
Over65              int64
55-64               int64
Under25             int64
over65_%_16       float64
over65_%_00       float64
over65_%change    float64
NET_LaborTotal      int64
%_LaborTotal      float64
NET_PrimeAge        int64
%_PrimeAge        float64
NET_Over65          int64
%_Over65          float64
NET_55-64           int64
%_55-64           float64
NET_Under25         int64
%_Under25         float64
dtype: object

In [12]:
Labor['STCO'] = Labor['STCO'].astype(str)
Labor['STCO'] = Labor['STCO'].apply(lambda x: '{0:0>5}'.format(x))

Labor.dtypes[:1]

STCO    object
dtype: object

----

# Labor Force by Places
_Identifying City vs Suburb trends_

## 2017

In [13]:
# Labor Force data for all counties in the US, 2016
Labor17_place = pd.read_json('https://api.census.gov/data/2016/acs/acs5?get='+
                              Labor17str +
                             ',NAME&for=place:*&in=state:*')
Labor17_place.columns = Labor17_place.iloc[0]
Labor17_place = Labor17_place[1:]

Labor17_place['state'] = Labor17_place['state'].apply(lambda x: '{0:0>2}'.format(x))
Labor17_place['place'] = Labor17_place['place'].apply(lambda x: '{0:0>3}'.format(x))
Labor17_place['STPL'] = Labor17_place[['state', 'place']].apply(lambda x: ''.join(x), axis=1)

# converting dtypes to int
for col in Labor17vars:
    Labor17_place[col] = Labor17_place[col].astype(int)

# aggregating age groups and calculations
Labor17_place['LaborForce'] = Labor17_place['B23025_001E'] - Labor17_place['B23025_007E']
Labor17_place['PrimeAge'] = Labor17_place['B23001_025E'] + Labor17_place['B23001_032E'] +\
                            Labor17_place['B23001_039E'] + Labor17_place['B23001_046E'] +\
                            Labor17_place['B23001_111E'] + Labor17_place['B23001_118E'] +\
                            Labor17_place['B23001_125E'] + Labor17_place['B23001_132E']
Labor17_place['Over65'] = Labor17_place['B23001_074E'] + Labor17_place['B23001_079E'] +\
                          Labor17_place['B23001_084E'] + Labor17_place['B23001_160E'] +\
                          Labor17_place['B23001_165E'] + Labor17_place['B23001_170E']
Labor17_place['55-64'] = Labor17_place['B23001_053E'] + Labor17_place['B23001_060E'] +\
                         Labor17_place['B23001_067E'] + Labor17_place['B23001_139E'] +\
                         Labor17_place['B23001_146E'] + Labor17_place['B23001_153E']
Labor17_place['Under25'] = Labor17_place['LaborForce'] - Labor17_place['PrimeAge'] -\
                           Labor17_place['Over65'] - Labor17_place['55-64']

Labor17_place = Labor17_place.drop(['state', 'place', 'B23025_001E', 'B23025_007E',
                                    'B23001_025E', 'B23001_032E', 'B23001_039E',
                                    'B23001_046E', 'B23001_111E', 'B23001_118E', 
                                    'B23001_125E', 'B23001_132E', 'B23001_074E',
                                    'B23001_079E', 'B23001_084E', 'B23001_160E',
                                    'B23001_139E', 'B23001_146E', 'B23001_153E',
                                    'B23001_053E', 'B23001_060E', 'B23001_067E',
                                    'B23001_165E', 'B23001_170E'], axis=1)
Labor17_place.columns = ['Name', 'STPL', 'LaborForce', 'PrimeAge', 
                         'Over65', '55-64', 'Under25']

print(Labor17_place.shape)
print(Labor17_place.dtypes)
Labor17_place.head()

(29574, 7)
Name          object
STPL          object
LaborForce     int64
PrimeAge       int64
Over65         int64
55-64          int64
Under25        int64
dtype: object


,Name,STPL,LaborForce,PrimeAge,Over65,55-64,Under25
1,"Abanda CDP, Alabama",0100100,74,34,0,40,0
2,"Abbeville city, Alabama",0100124,855,466,56,204,129
3,"Adamsville city, Alabama",0100460,2107,1314,104,425,264
4,"Addison town, Alabama",0100484,317,184,14,60,59
5,"Akron town, Alabama",0100676,93,56,2,11,24


## 2000

In [14]:
# Labor Force data for all counties in the US, 2000
Labor00_place = pd.read_json('https://api.census.gov/data/2000/sf3?get='+
                              Labor00str +
                              ',NAME&for=place:*&in=state:*')
Labor00_place.columns = Labor00_place.iloc[0]
Labor00_place = Labor00_place[1:]

Labor00_place['state'] = Labor00_place['state'].apply(lambda x: '{0:0>2}'.format(x))
Labor00_place['place'] = Labor00_place['place'].apply(lambda x: '{0:0>3}'.format(x))
Labor00_place['STPL'] = Labor00_place[['state', 'place']].apply(lambda x: ''.join(x), axis=1)

# converting dtypes to int
for col in Labor00vars:
    Labor00_place[col] = Labor00_place[col].astype(int)

Labor00_place['LaborForce00'] = Labor00_place['P043003'] + Labor00_place['P043010']
Labor00_place['PrimeAge00'] = Labor00_place['PCT035025'] + Labor00_place['PCT035032'] +\
                              Labor00_place['PCT035039'] + Labor00_place['PCT035046'] +\
                              Labor00_place['PCT035117'] + Labor00_place['PCT035124'] +\
                              Labor00_place['PCT035131'] + Labor00_place['PCT035138']
Labor00_place['Over65_00'] = Labor00_place['PCT035074'] + Labor00_place['PCT035081'] +\
                             Labor00_place['PCT035088'] + Labor00_place['PCT035166'] +\
                             Labor00_place['PCT035173'] + Labor00_place['PCT035180']
Labor00_place['55-64_00'] = Labor00_place['PCT035053'] + Labor00_place['PCT035060'] +\
                            Labor00_place['PCT035067'] + Labor00_place['PCT035145'] +\
                            Labor00_place['PCT035152'] + Labor00_place['PCT035159']
Labor00_place['Under25_00'] = Labor00_place['LaborForce00'] - Labor00_place['PrimeAge00'] -\
                              Labor00_place['Over65_00'] - Labor00_place['55-64_00']

Labor00_place = Labor00_place.drop(['state', 'place', 'P043003', 'P043010',
                                    'PCT035025', 'PCT035032', 'PCT035039',
                                    'PCT035046', 'PCT035117', 'PCT035124', 
                                    'PCT035131', 'PCT035138', 'PCT035074',
                                    'PCT035081', 'PCT035088', 'PCT035166',
                                    'PCT035053', 'PCT035060', 'PCT035067',
                                    'PCT035145', 'PCT035152', 'PCT035159',
                                    'PCT035173', 'PCT035180'], axis=1)
Labor00_place.columns = ['Name', 'STPL', 'LaborForce00', 'PrimeAge00',
                         'Over65_00', '55-64_00', 'Under25_00']

print(Labor00_place.shape)
print(Labor00_place.dtypes)
Labor00_place.head()

(25375, 7)
Name            object
STPL            object
LaborForce00     int64
PrimeAge00       int64
Over65_00        int64
55-64_00         int64
Under25_00       int64
dtype: object


,Name,STPL,LaborForce00,PrimeAge00,Over65_00,55-64_00,Under25_00
1,Abbeville city,0100124,1269,804,60,167,238
2,Adamsville city,0100460,2419,1706,93,310,310
3,Addison town,0100484,334,219,10,61,44
4,Akron town,0100676,178,142,3,16,17
5,Alabaster city,0100820,12609,10064,221,1034,1290


### Merging 2000 + 2017 data

In [15]:
Labor_place = Labor00_place.merge(Labor17_place, on='STPL')

Labor_place = Labor_place.drop(['Name_x'], axis=1)
Labor_place = Labor_place.append(Labor_place.sum(numeric_only=True), ignore_index=True)
Labor_place = Labor_place.set_index('Name_y').fillna(0).astype(int)

Labor_place['over65_%_16'] = Labor_place['Over65'] / Labor_place['LaborForce']
Labor_place['over65_%_00'] = Labor_place['Over65_00'] / Labor_place['LaborForce00']

Labor_place['over65_%change'] = Labor_place['over65_%_16'] - Labor_place['over65_%_00']

Labor_place['NET_LaborTotal'] = Labor_place['LaborForce'] - Labor_place['LaborForce00']
Labor_place['%_LaborTotal'] = (Labor_place['LaborForce'] - Labor_place['LaborForce00']) \
                               / Labor_place['LaborForce00']
Labor_place['NET_PrimeAge'] = Labor_place['PrimeAge'] - Labor_place['PrimeAge00']
Labor_place['%_PrimeAge'] = (Labor_place['PrimeAge'] - Labor_place['PrimeAge00']) \
                            / Labor_place['PrimeAge00']
Labor_place['NET_Over65'] = Labor_place['LaborForce'] - Labor_place['LaborForce00']
Labor_place['%_Over65'] = (Labor_place['Over65'] - Labor_place['Over65_00']) \
                     / Labor_place['Over65_00']
Labor_place['NET_55-64'] = Labor_place['55-64'] - Labor_place['55-64_00']
Labor_place['%_55-64'] = (Labor_place['55-64'] - Labor_place['55-64_00']) \
                          / Labor_place['55-64_00']
Labor_place['NET_Under25'] = Labor_place['Under25'] - Labor_place['Under25_00']
Labor_place['%_Under25'] = (Labor_place['Under25'] - Labor_place['Under25_00']) \
                            / Labor_place['Under25_00']


Labor_place.index = Labor_place.index.fillna('UStotal')

print(Labor_place.shape)
Labor_place.tail()

(24650, 24)


,STPL,LaborForce00,PrimeAge00,Over65_00,55-64_00,Under25_00,LaborForce,PrimeAge,Over65,55-64,...,NET_LaborTotal,%_LaborTotal,NET_PrimeAge,%_PrimeAge,NET_Over65,%_Over65,NET_55-64,%_55-64,NET_Under25,%_Under25
Name_y,,,,,,,,,,,,,,,,,,,,,
"Villalba zona urbana, Puerto Rico",7286831,1197,912,31,61,193,1494,1019,68,160,...,297,0.248120,107,0.117325,297,1.193548,99,1.622951,54,0.279793
"Yabucoa zona urbana, Puerto Rico",7287863,1458,1100,18,89,251,2226,1704,31,170,...,768,0.526749,604,0.549091,768,0.722222,81,0.910112,70,0.278884
"Yauco zona urbana, Puerto Rico",7288035,6064,4395,118,509,1042,5834,4475,181,778,...,-230,-0.037929,80,0.018203,-230,0.533898,269,0.528487,-642,-0.616123
"Yaurel comunidad, Puerto Rico",7288121,326,223,9,31,63,276,206,0,27,...,-50,-0.153374,-17,-0.076233,-50,-1.000000,-4,-0.129032,-20,-0.317460
UStotal,0,98868078,69253950,3292657,9460639,16860832,116600623,76194405,5423319,17452708,...,17732545,0.179356,6940455,0.100217,17732545,0.647095,7992069,0.844771,669359,0.039699
